In [2]:
import cv2
import numpy as np
import PySimpleGUI as sg
import pickle
def carModel(frame):

    pick = open('modelCar.sav', 'rb')
    model = pickle.load(pick)
    pick.close()

    carPhoto = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    carPhoto2 = cv2.resize(carPhoto, (50, 50))
    carPhoto3 = np.array(carPhoto2).flatten()

    prediction = model.predict([carPhoto3])
    categories = ['front', 'Back']
    result = categories[prediction[0]]

    if result == "front":
        return False
    else:
        return True
    
def carCrossesAtRedLight(filename):

    cap = cv2.VideoCapture(filename)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter('video_car_violation.avi', fourcc, 25, (1280, 720))

    car_cascade = cv2.CascadeClassifier('cars.xml')

    car_violation = [[0, 0, 0, 0]]
    video_save = False
    delta =50

    while (True):

        ret, frame = cap.read()

        if ret == True:

            crop_img = frame[0:600, 350:1000]
            gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            mask_for_tl = create_mask(crop_img)

            red_circles = cv2.HoughCircles(mask_for_tl, cv2.HOUGH_GRADIENT, 1, 80, param1=50, param2=10, minRadius=0,
                                           maxRadius=15)

            if red_circles is not None:
                red_circles = np.uint16(np.around(red_circles))

                for i in red_circles[0, :]:

                    #Draw a circle
                    #cv2.circle(crop_img, (i[0], i[1]), i[2] + 10, (0, 0, 0), 2)

                    cars = car_cascade.detectMultiScale(image=gray, scaleFactor=2.7, minNeighbors=1, minSize=(80, 80))

                    #Draw a rectangle
                    #for (x, y, w, h) in cars:
                        #cv2.rectangle(crop_img, (x, y), (x + w, y + h), (0, 0, 255), 2)

                    if car_violation[0][0] == 0 and type(cars) != tuple:
                        car_violation = cars

                    if type(cars) != tuple:
                        result = carModel(crop_img)

                        if result == True:

                            if cars[0][0] - car_violation[0][0] > delta or cars[0][0] - car_violation[0][0] < -1*delta:
                                car_violation = cars
                                video_save = True

            if video_save == True:
                video.write(frame)
            #To show the video
            #cv2.imshow('detected results', crop_img)

        else:
            break

        # Wait for Esc key to stop
        #if cv2.waitKey(33) == 27:
        #    break


    cap.release()
    #cv2.destroyAllWindows()

    return video_save


def create_mask(img):

    # color range
    l_red1 = np.array([0, 100, 100])
    u_red1 = np.array([10, 255, 255])
    l_red2 = np.array([160, 100, 100])
    u_red2 = np.array([180, 255, 255])

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv, l_red1, u_red1)
    mask2 = cv2.inRange(hsv, l_red2, u_red2)
    maskRed = cv2.add(mask1, mask2)
    maskRed = cv2.erode(maskRed, None, iterations=2)
    maskRed = cv2.dilate(maskRed, None, iterations=2)

    return maskRed


if '_main_' == '_main_':

    filename = sg.popup_get_file('Enter the file you wish to process')
    sg.popup('You entered', filename)

    if filename == None:
        print("NO video file")
        check = False
    else:
        check = True
    if check:    
        print("The program goes through the video and checks if a vehicle has passed a red light")

        save_video = carCrossesAtRedLight(filename)

        if save_video == True:
            print("Car passed in red traffic light")
            print("The video save in the directory where the main file is located")
        else:
            print("No vehicle passed a red light")
    else:
        exit()
    

The program goes through the video and checks if a vehicle has passed a red light


/home/tal/.local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/tal/.local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/tal/.local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/tal/.local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.24.1 when using version 0.24.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.wa

Car passed in red traffic light
The video save in the directory where the main file is located
